In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
24,model_4_9_10,0.998969,0.997138,0.996476,0.997049,0.124363,0.467676,0.207207,0.345102,"Hidden Size=[25], regularizer=0.1, learning_ra..."
25,model_4_9_9,0.998967,0.997204,0.996786,0.997172,0.124600,0.456761,0.188967,0.330740,"Hidden Size=[25], regularizer=0.1, learning_ra..."
26,model_4_9_11,0.998965,0.997072,0.996184,0.996932,0.124801,0.478331,0.224344,0.358808,"Hidden Size=[25], regularizer=0.1, learning_ra..."
27,model_4_9_12,0.998958,0.997009,0.995912,0.996821,0.125721,0.488595,0.240355,0.371776,"Hidden Size=[25], regularizer=0.1, learning_ra..."
28,model_4_9_8,0.998957,0.997272,0.997114,0.997299,0.125758,0.445784,0.169698,0.315861,"Hidden Size=[25], regularizer=0.1, learning_ra..."
29,model_4_9_13,0.998947,0.996950,0.995659,0.996717,0.126978,0.498390,0.255238,0.383965,"Hidden Size=[25], regularizer=0.1, learning_ra..."
30,model_4_9_7,0.998937,0.997338,0.997457,0.997429,0.128144,0.434995,0.149514,0.300651,"Hidden Size=[25], regularizer=0.1, learning_ra..."
31,model_4_9_14,0.998935,0.996893,0.995425,0.996619,0.128456,0.507645,0.269022,0.395352,"Hidden Size=[25], regularizer=0.1, learning_ra..."
32,model_4_9_15,0.998922,0.996840,0.995208,0.996529,0.130067,0.516337,0.281743,0.405940,"Hidden Size=[25], regularizer=0.1, learning_ra..."
33,model_1_9_0,0.998917,0.999974,0.995236,0.998523,0.130632,0.003758,0.348354,0.165921,"Hidden Size=[30], regularizer=0.01, learning_r..."
